# 모듈 import

In [1]:
import sys
import random
import pickle

import pandas as pd
import numpy as np
import json
from konlpy.tag import Twitter
from sklearn.externals import joblib
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB , GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [2]:
import json
import os
import random

import konlpy

#import elice_challenge as ec

# 파일 읽기

In [3]:
test_out_data = open('/elice/data/test.output')
#test_out_data = test_out_data.read()
#3test_out_data = test_out_data.splitlines()

In [4]:
train_data = json.load(open('/elice/data/train_data.json'))

test_data = json.load(open('/elice/data/test.input.json'))

with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out_data = test_out.splitlines()

grading_data = json.load(open('/elice/data/grading.input.json'))

파일이 제대로 읽혔는지 확인합니다.

In [8]:
train_df = pd.DataFrame(train_data)[['text','writing']]
test_df = pd.DataFrame(test_data)[['text']]
test_df['writing'] = test_out_data
grading_df = pd.DataFrame(grading_data)[['text']]

# vectorize

In [9]:
X = train_df.text
y = train_df.writing

In [10]:
# 처음엔 이걸로 # vectorize train using Tfid(bag of words)

twitter = Twitter()

def tokenize_pos(doc):
    return ['/'.join(t) for t in twitter.pos(doc, norm=True, stem=True)]

vectorizer =  TfidfVectorizer(tokenizer=tokenize_pos, ngram_range=(1,2))

X = vectorizer.fit_transform(X)

### Fitting train data

In [13]:
# fitting train data with classifier

#model = SGDClassifier(alpha=1.9e-6, n_iter=19).fit(X, y)
model = LogisticRegression(C=15800).fit(X,y)
#model = xgboost.XGBClassifier().fit(X,y)
#model = MultinomialNB(alpha=5).fit(X,y)
#model = ExtraTreesClassifier(n_estimators=10).fit(X,y)

In [12]:
# function for predicting test data

def testX_to_vec(X):
    feature_list = vectorizer.get_feature_names()

    test_vectorizer =  TfidfVectorizer(tokenizer=tokenize_pos, ngram_range=(1,3), vocabulary = feature_list)
    X_test = test_vectorizer.fit_transform(X)
    
    return X_test


def predict_rank(X_test):
    
    test_prob = model.predict_proba(X_test)

    test_class = pd.DataFrame((-test_prob).argsort())
    
    class_df = pd.DataFrame(model.classes_).reset_index()
    
    # data frame 생성
    for i in range(len(model.classes_)):
        class_df.columns = [i, 'rank_'+str(i)]
        test_class = pd.merge(test_class, class_df, on=i, how='left')

    # rank columns    
    rank_columns = ['rank_0',  'rank_1',  'rank_2',  'rank_3',  'rank_4',
                    'rank_5',  'rank_6',  'rank_7',  'rank_8',  'rank_9', 
                    'rank_10', 'rank_11', 'rank_12', 'rank_13', 'rank_14', 
                    'rank_15', 'rank_16', 'rank_17', 'rank_18']

    #dataframe to csv
    test_class_csv = test_class[rank_columns].to_csv(header=False, index=False)
    test_class_csv = test_class_csv.splitlines()
    
    return test_class_csv



def mrr(rank_class, test_out_data):

    pred = []
    for i in range(len(rank_class)):
        pred.append(rank_class[i].split(','))
    
    position = np.zeros([len(rank_class),1])
    pred = np.array(pred)
    test_true = np.array(test_out_data)
    
    for i in range(len(rank_class)):
        position[i] = np.where(pred[i]==test_true[i])[0]+1
    
    return ((1/position).sum())/len(rank_class)

In [14]:
test_X = test_df.text
#test to vec
test_X = testX_to_vec(test_X)

In [15]:
rank_class = predict_rank(test_X)
mrr(rank_class, test_out_data)

0.93939857516733172

### Predict grading_data

In [16]:
#grading to vec
X_grading = grading_df['text']
X_grading = testX_to_vec(X_grading)

In [17]:
# predict rank
grading_rank_class = predict_rank(X_grading) # test score 0.87846

In [18]:
# write csv
output_file = 'submission.txt'
with open(output_file, 'w') as fp:
    for row in grading_rank_class:
        fp.write(row)
        fp.write('\n')

#### Submission

In [20]:
import elice_challenge as ec

In [25]:
ec.upload()

이전에 업로드 된 파일 삭제 중...

두 종류의 파일들을 제출해야 합니다.
 - submission.txt : grading.input.json 에 해당하는 출력값
 - submission.ipynb 혹은 submission.py : submission.txt를 재현가능한 소스 코드
제출할 파일들은 comma (,) 로 나누어 써서 제출할 수 있습니다.
제출 예제) submission.ipynb,submission.txt,additional_data.json

> submission.txt,classify_article.ipynb
submission.txt 체크 중...
submission.txt: 2016000 bytes
classify_article.ipynb 체크 중...
classify_article.ipynb: 10212 bytes
업로드 중: submission.txt
업로드 중: classify_article.ipynb
업로드가 완료되었습니다.
submit() 으로 점수를 확인할 수 있습니다.


업로드가 완료되면 제출을 시도해봅니다.

In [23]:
ec.submit()

submission 데이터 개수 = 9000
답안 데이터 개수 = 9000
MRR: 0.94435
Score: 94 pts.
94
Score: 94
